# Triadicity operators: computations for the paper
This notebook reproduces Tables and Figures in `triadicity_jcn.tex` from `Glist.p`.


In [ ]:

import os, pickle, math
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

DATA_PATH = "Glist.p"   # adjust if needed
OUT_DIR = "paper_outputs"
TABLES_DIR = os.path.join(OUT_DIR, "tables")
FIGS_DIR = os.path.join(OUT_DIR, "figs")
os.makedirs(TABLES_DIR, exist_ok=True)
os.makedirs(FIGS_DIR, exist_ok=True)

with open(DATA_PATH, "rb") as f:
    Glist = pickle.load(f)

len(Glist), Glist[0][0]


In [ ]:

def triangles_dict(G):
    tri = nx.triangles(G)
    tau = sum(tri.values())//3
    return tri, tau

def m2(G):
    return sum(d*(d-1)//2 for _, d in G.degree())

def traversing_types(G, tri, S):
    Vc = set(v for v in G if tri[v] > 0)
    Vt = set(G) - Vc
    dominated = Vc - set(S)
    T1, T2, T3, T4 = set(), set(), set(), set()
    for v in Vt:
        adjS = any(G.has_edge(v, s) for s in S)
        adjD = any(G.has_edge(v, u) for u in dominated)
        adjC = any(G.has_edge(v, u) for u in Vc)
        if adjS and not adjD:
            T1.add(v)
        elif adjS and adjD:
            T2.add(v)
        elif (not adjS) and adjD:
            T3.add(v)
        elif not adjC:
            T4.add(v)
        else:
            T3.add(v)
    return T1, T2, T3, T4, Vc, Vt, dominated

def build_partition(G, tri, S):
    T1, T2, T3, T4, Vc, Vt, dominated = traversing_types(G, tri, S)
    S_list = sorted(list(S), key=str)
    blocks = {f"EGO:{s}": set([s]) for s in S_list}

    for v in sorted(dominated, key=str):
        candidates = [s for s in S_list if G.has_edge(v, s)]
        s = candidates[0] if candidates else S_list[0]
        blocks[f"EGO:{s}"].add(v)

    for v in sorted(T1 | T2, key=str):
        candidates = [s for s in S_list if G.has_edge(v, s)]
        s = candidates[0] if candidates else S_list[0]
        blocks[f"EGO:{s}"].add(v)

    for v in sorted(T3 | T4, key=str):
        blocks[f"TR:{v}"] = set([v])

    return blocks, (T1, T2, T3, T4)

def block_order(blocks):
    egos = sorted([b for b in blocks if b.startswith("EGO:")])
    trs = sorted([b for b in blocks if b.startswith("TR:")])
    return egos + trs

def quotient_directed_matrix(G, blocks, order):
    bidx = {b: i for i, b in enumerate(order)}
    node_to_block = {v: b for b, vs in blocks.items() for v in vs}
    r = len(order)
    B = np.zeros((r, r), dtype=float)
    for u, v in G.edges():
        i, j = bidx[node_to_block[u]], bidx[node_to_block[v]]
        if i == j:
            B[i, i] += 2
        else:
            B[i, j] += 1
            B[j, i] += 1
    return B

def aggregate_two_walks(G, blocks, order):
    nodes = list(G.nodes())
    idx = {n: i for i, n in enumerate(nodes)}
    n = len(nodes)
    A = np.zeros((n, n))
    for u, v in G.edges():
        i, j = idx[u], idx[v]
        A[i, j] = 1
        A[j, i] = 1
    A2 = A @ A

    bidx = {b: i for i, b in enumerate(order)}
    R = np.zeros((n, len(order)))
    for b, vs in blocks.items():
        j = bidx[b]
        for v in vs:
            R[idx[v], j] = 1

    return R.T @ A2 @ R


In [ ]:

orig_rows = []
quot_rows = []
ratios = []

for title, gid, G, dicts, dom in Glist:
    tri, tau = triangles_dict(G)
    m2v = m2(G)
    omega = m2v - 3*tau
    Vc = sum(1 for v in G if tri[v] > 0)
    Vt = G.number_of_nodes() - Vc

    blocks, types = build_partition(G, tri, dom)
    order = block_order(blocks)
    B = quotient_directed_matrix(G, blocks, order)
    M = aggregate_two_walks(G, blocks, order)

    B2 = B @ B
    np.fill_diagonal(B2, 0.0)
    np.fill_diagonal(M, 0.0)
    rho = (M.sum() / B2.sum()) if B2.sum() > 0 else np.nan
    ratios.append((gid, rho))

    T1, T2, T3, T4 = types
    orig_rows.append(dict(Graph=gid, n=G.number_of_nodes(), m=G.number_of_edges(),
                          triangles=tau, m2=m2v, omega=int(omega),
                          Vc=Vc, Vt=Vt, dom=len(dom)))

    quot_rows.append(dict(Graph=gid, blocks=len(order), egoblocks=len(dom),
                          TR_singletons=len(T3)+len(T4),
                          B_edges=int((B - np.diag(np.diag(B))).sum()/2),
                          B_internal=int(np.trace(B)/2),
                          ratio=float(rho)))

orig_df = pd.DataFrame(orig_rows)
quot_df = pd.DataFrame(quot_rows)
orig_df, quot_df.head()


In [ ]:

# Export LaTeX tables
orig_tex = orig_df.to_latex(index=False, escape=True,
                            caption="Summary invariants for the ten base graphs.",
                            label="tab:base",
                            column_format="lrrrrrrrr")
with open(os.path.join(TABLES_DIR, "base_summary.tex"), "w") as f:
    f.write(orig_tex)

quot_df2 = quot_df.copy()
quot_df2["ratio"] = quot_df2["ratio"].map(lambda x: f"{x:.3f}" if pd.notna(x) else "NA")
quot_tex = quot_df2.to_latex(index=False, escape=True,
                             caption="Quotient/contraction diagnostics for the ego--traversing partition (directed edge-sum quotient).",
                             label="tab:quot",
                             column_format="lrrrrrrl")
with open(os.path.join(TABLES_DIR, "quotient_summary.tex"), "w") as f:
    f.write(quot_tex)

print("Wrote tables to:", TABLES_DIR)


In [ ]:

# Figure 1: ECDFs of local clustering coefficients
plt.figure()
for title, gid, G, dicts, dom in Glist:
    Cvals = np.array(list(nx.clustering(G).values()))
    xs = np.sort(Cvals)
    ys = np.arange(1, len(xs)+1)/len(xs)
    plt.step(xs, ys, where="post", label=gid, linewidth=1)
plt.xlabel("local clustering coefficient C(v)")
plt.ylabel("ECDF")
plt.legend(fontsize=6, ncol=2)
plt.tight_layout()
plt.savefig(os.path.join(FIGS_DIR, "ecdf_clustering.pdf"))
plt.close()

# Figure 2: ratio bar plot
rat_df = pd.DataFrame(ratios, columns=["Graph","ratio"]).sort_values("ratio")
plt.figure()
plt.bar(rat_df["Graph"], rat_df["ratio"])
plt.xticks(rotation=45, ha="right")
plt.ylabel(r"$\sum_{a\neq b} M_{ab} \,/\, \sum_{a\neq b} (B^2)_{ab}$")
plt.tight_layout()
plt.savefig(os.path.join(FIGS_DIR, "quotient_ratio.pdf"))
plt.close()

print("Wrote figures to:", FIGS_DIR)
rat_df
